## 第十章 模式识别系统的评价

这一章介绍了大多数其他教材放在第一章的内容，即分类算法的检验方法。

教材的重点放在了检验方法得出的错误率的理论分析上面，即从概率论的角度分析错误率的期望和方差。这部分都是数学推导，不需要写代码。

而教材中提到的计算错误率的检验方法实现起来都很非常简单，而且在sklearn中都有库可以很方便的调用。

只有扰动重采样估计SVM错误率的置信区间非常复杂，但是这个算法其实不重要，作者把它放在教材上主要还是因为这个算法是作者自己发的文章。

下面我只实现一下bootstrap .632估计。

In [1]:
from sklearn.utils import resample
from sklearn.svm import LinearSVC
import numpy as np

下面用手写数字识别的数据集。

In [2]:
X = np.load("./data/digit_0_1_X.npy")
y = np.load("./data/digit_0_1_y.npy")
X = X[:1000]
y = y[:1000] #前一千个是0和1，后面是其他数字
y = y.reshape(1000)

先计算自举错误率B1。

In [3]:
# 下面计算十次bootstrap样本集在线性支持向量机上的错误率
b1 = 0
for i in range(10):
    bootstrap_index = []
    test_index = set(range(X.shape[0]))
    X_bootstrap = resample(X)
    for x in X_bootstrap:
        for j in range(X.shape[0]):
            if (X[j] == x).all():
                bootstrap_index.append(j)
                if j in test_index:
                    test_index.remove(j)
                break
    test_index = list(test_index)
    y_bootstrap = y[bootstrap_index]
    X_test = X[test_index]
    y_test = y[test_index]
    print("bootstrap第", i + 1, "次，自举样本集中包含了原始样本集",
          (1 - len(test_index) / X.shape[0]) * 100, "%的样本。", sep="")
    lsvc = LinearSVC()
    lsvc.fit(X_bootstrap, y_bootstrap)
    err = 1 - lsvc.score(X_test, y_test)
    b1 += err
    print("本次训练错误率为：", err,sep="")
b1 /= 10
print("自举平均错误率B1 =", b1)

bootstrap第1次，自举样本集中包含了原始样本集67.10000000000001%的样本。
本次训练错误率为：0.003039513677811523
bootstrap第2次，自举样本集中包含了原始样本集63.2%的样本。
本次训练错误率为：0.0027173913043477826
bootstrap第3次，自举样本集中包含了原始样本集61.5%的样本。
本次训练错误率为：0.0051948051948051965
bootstrap第4次，自举样本集中包含了原始样本集62.7%的样本。
本次训练错误率为：0.002680965147453085
bootstrap第5次，自举样本集中包含了原始样本集63.4%的样本。
本次训练错误率为：0.002732240437158473
bootstrap第6次，自举样本集中包含了原始样本集62.7%的样本。
本次训练错误率为：0.00536193029490617
bootstrap第7次，自举样本集中包含了原始样本集63.7%的样本。
本次训练错误率为：0.0027548209366391463
bootstrap第8次，自举样本集中包含了原始样本集64.7%的样本。
本次训练错误率为：0.0028328611898017497
bootstrap第9次，自举样本集中包含了原始样本集62.8%的样本。
本次训练错误率为：0.0
bootstrap第10次，自举样本集中包含了原始样本集63.2%的样本。
本次训练错误率为：0.0027173913043477826
自举平均错误率B1 = 0.003003191948727091


再计算全部样本上的训练错误率（视在错误率）。

In [4]:
lsvc = LinearSVC()
lsvc.fit(X, y)
ae = 1 - lsvc.score(X, y)
print("全部样本上的训练错误率AE =", ae)

全部样本上的训练错误率AE = 0.0


接下来计算B.632错误率。

In [5]:
print("B.632错误率 =", 0.368 * ae + 0.632 * b1)

B.632错误率 = 0.0018980173115955214
